# Filter Dataset and Train a PyTorch Model
In this Jupyter notebook, we will explore the implementation of a modified version of the `ImageFolder` dataset from the PyTorch `torchvision` package. This modified dataset filters out samples whose filenames are listed in a given CSV file. You obtain the CSV file by running fastdup (see [this notebook](./analyze.ipynb)) or dropping us an email at info@visual-layer.com .

<!--<badge>--><a href="https://colab.research.google.com/github/visual-layer/vl-datasets/blob/master/notebooks/train-clean-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

## Installation & Setting Up

First, we need to install the fastdup and matplotlib libraries. Run the following command in your Jupyter notebook to install them.

In [ ]:
!pip install -U torch torchvision pandas

## Download foods-101 Dataset
Next, we need to download the dataset. For this tutorial, we will use the foods-101 dataset. Run the following commands in your Jupyter notebook to download and extract the dataset:

In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xf food-101.tar.gz

Once you're done extracting the resulting directory should look like this

```
food-101/
├── images
│   ├── apple_pie
│   │   ├── 0001.jpg
│   │   └── 0002.jpg
│   ├── baby_back_ribs
│   ├── baklava
│   ├── ...
│   ├── ...
│   ├── ...
│   └── waffles
├── meta
|   ├── classes.txt
│   ├── labels.txt
│   ├── test.json
│   ├── test.txt
│   ├── train.json
│   └── train.txt
├── license_agreement.txt
└── README.txt
```

The train-test split information is contained in the `meta/` folder.

## Structuring Data

Let's take the above folder structure and turn it into a structure we can readily use with Torchvision.
We will reorganize the above into the following folder structure:

```
food-101/
├── train
│   ├── apple_pie
│   │   ├── 0001.jpg
│   │   └── 0002.jpg
│   ├── baby_back_ribs
│   ├── baklava
│   ├── ...
│   ├── ...
│   ├── ...
│   └── waffles
├── test
│   ├── apple_pie
│   │   ├── 01000.jpg
│   │   └── 01001.jpg
│   ├── baby_back_ribs
│   ├── baklava
│   ├── ...
│   ├── ...
│   ├── ...
│   └── waffles
├── meta
|   ├── classes.txt
│   ├── labels.txt
│   ├── test.json
│   ├── test.txt
│   ├── train.json
│   └── train.txt
├── license_agreement.txt
└── README.txt

```

In order to do that let's run the following code.

In [ ]:
import shutil
import collections
from collections import defaultdict
import os

class_to_ix = {}
ix_to_class = {}
with open('food-101/meta/classes.txt', 'r') as txt:
    classes = [l.strip() for l in txt.readlines()]
    class_to_ix = dict(zip(classes, range(len(classes))))
    ix_to_class = dict(zip(range(len(classes)), classes))
    class_to_ix = {v: k for k, v in ix_to_class.items()}
sorted_class_to_ix = collections.OrderedDict(sorted(class_to_ix.items()))

# Only split files if haven't already
if not os.path.isdir('./food-101/test') and not os.path.isdir('./food-101/train'):

    def copytree(src, dst, symlinks = False, ignore = None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    def generate_dir_file_map(path):
        dir_files = defaultdict(list)
        with open(path, 'r') as txt:
            files = [l.strip() for l in txt.readlines()]
            for f in files:
                dir_name, id = f.split('/')
                dir_files[dir_name].append(id + '.jpg')
        return dir_files

    train_dir_files = generate_dir_file_map('food-101/meta/train.txt')
    test_dir_files = generate_dir_file_map('food-101/meta/test.txt')


    def ignore_train(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = train_dir_files[subdir]
        return to_ignore

    def ignore_test(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = test_dir_files[subdir]
        return to_ignore

    copytree('food-101/images', 'food-101/test', ignore=ignore_train)
    copytree('food-101/images', 'food-101/train', ignore=ignore_test)
    
else:
    print('Train/Test files already copied into separate folders.')

## Imports
We will use the following libraries in this tutorial. Import them in your Jupyter notebook by running the following commands:

In [1]:
from pathlib import Path

import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

import torchvision
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

## Define the preprocessing transforms
We define the preprocessing transforms for the dataset. We have two transforms: `train_transform` and `valid_transform`.

In [2]:
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

valid_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

## Define a custom class FilteredDataset
We define a custom class `FilteredDataset` that extends `ImageFolder` class. 
This class will allow us to exclude files from the dataset with a `.csv` file. 

Here is the code for the `FilteredDataset` class:

In [3]:
class FilteredDataset(ImageFolder):
    """
    A modified version of torchvision.datasets.ImageFolder that filters out samples whose filenames
    are listed in a given CSV file.

    See: https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html

    Args:
        root_dir (string): Root directory path of the dataset.
        csv_path (string, optional): Path to a CSV file containing a list of excluded filenames.
                                     Default: None.
        transform (callable, optional): A function/transform that takes in a PIL image and returns a
                                         transformed version. E.g, ``transforms.RandomCrop``
                                         Default: None.
        target_transform (callable, optional): A function/transform that takes in the target and
                                                transforms it. Default: None.

    Example usage:

        # Load the dataset and exclude certain samples
        dataset = FilteredDataset("dataset/images", "files-to-exclude.csv", transform=transforms.ToTensor())

        # Create a dataloader
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    """

    def __init__(self, root_dir, csv_path=None, transform=None, target_transform=None):
        root_dir = Path(root_dir)
        super().__init__(root_dir, transform=transform, target_transform=target_transform)

        if csv_path:
            self.excluded_files = pd.read_csv(csv_path, header=0)
            self.excluded_files['filename'] = self.excluded_files['filename'].apply(lambda x: str(root_dir) + "/" + x )
            self.excluded_filenames = set(self.excluded_files['filename'])
 
            print(f"Original Samples: {len(self.samples)} in {root_dir}")
            print(f"Excluded: {len(self.excluded_filenames)} in {root_dir}")
            excluded_indices = [i for i, (path, _) in enumerate(self.samples) if path in self.excluded_filenames]
            self.samples = [sample for i, sample in enumerate(self.samples) if i not in excluded_indices]
            self.targets = [target for i, target in enumerate(self.targets) if i not in excluded_indices]
            print(f"Cleaned Samples: {len(self.samples)} in {root_dir}")

## Exclude files
Using the custom `FilteredDataset` class, we can conveniently exclude the files specified in the `.csv` files from being loaaded into the dataset.

In [4]:
train_dataset = FilteredDataset("food-101/train", "food_101_vl-datasets_analysis.csv", transform=train_transform)

Original Samples: 75750 in food-101/train
Excluded: 498 in food-101/train
Cleaned Samples: 75343 in food-101/train


In [5]:
valid_dataset = FilteredDataset("food-101/test", "food_101_vl-datasets_analysis.csv", transform=train_transform)

Original Samples: 25250 in food-101/test
Excluded: 498 in food-101/test
Cleaned Samples: 25159 in food-101/test


We can also view the exclude files with:

In [6]:
train_dataset.excluded_files

,filename,reason,value,prototype
0,food-101/train/apple_pie/1487150.jpg,Duplicate,0.9662,apple_pie/1486972.jpg
1,food-101/train/apple_pie/3324492.jpg,Duplicate,0.9817,apple_pie/2106005.jpg
2,food-101/train/apple_pie/3670966.jpg,Duplicate,0.9879,apple_pie/3670548.jpg
3,food-101/train/apple_pie/839845.jpg,Duplicate,0.9964,apple_pie/839808.jpg
4,food-101/train/baby_back_ribs/2306066.jpg,Duplicate,0.9862,baby_back_ribs/2306008.jpg
...,...,...,...,...
508,food-101/train/sashimi/241368.jpg,Dark,15.7813,NaN
509,food-101/train/scallops/3314913.jpg,Dark,13.7173,NaN
510,food-101/train/spring_rolls/182658.jpg,Dark,8.9502,NaN
511,food-101/train/bread_pudding/444890.jpg,File-Size,9715.0000,NaN


In [7]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=True, pin_memory=True)

## Define the model architecture
Let's construct a basic convolutional model, Resnet18 from Torchvision.

In [8]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

## Define the loss function and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train the model
Now, let's write a simple training loop to train the model for 10 epochs on a GPU or CPU.

In [10]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")


Using device: cuda
Epoch 1 - Loss: 2.3990132137880487
Epoch 2 - Loss: 1.8088141485796136
Epoch 3 - Loss: 1.598961140341678
Epoch 4 - Loss: 1.4813383158990892
Epoch 5 - Loss: 1.3898559869345972
Epoch 6 - Loss: 1.3187752719652854
Epoch 7 - Loss: 1.258394802626917
Epoch 8 - Loss: 1.1980765166929213
Epoch 9 - Loss: 1.1490829083879115
Epoch 10 - Loss: 1.1118874396307994


## Evaluate the model
Finally we evaluate the model on the validation set and prints it's accuracy.

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}")


Accuracy: 69.50594220755993
